# 02. M�dulos y Principios SOLID
Encapsulamiento, responsabilidad �nica e inyecci�n de dependencias.

In [ ]:
// S - Single Responsibility: Separar l�gica de negocio de presentaci�n

class RegistroAcademico {
    #estudiantes = new Map(); // Encapsulamiento privado
    
    agregarEstudiante(id, nombre, carrera) {
        if (this.#estudiantes.has(id)) {
            throw new Error(`Estudiante ${id} ya existe`);
        }
        this.#estudiantes.set(id, {id, nombre, carrera, notas: []});
        return true;
    }
    
    agregarNota(idEstudiante, materia, calificacion) {
        const est = this.#estudiantes.get(idEstudiante);
        if (!est) throw new Error('Estudiante no existe');
        if (calificacion < 0 || calificacion > 100) throw new Error('Calificaci�n inv�lida');
        
        est.notas.push({materia, calificacion, fecha: new Date().toISOString()});
        return this.calcularPromedio(idEstudiante);
    }
    
    calcularPromedio(idEstudiante) {
        const est = this.#estudiantes.get(idEstudiante);
        if (!est || est.notas.length === 0) return 0;
        const suma = est.notas.reduce((acc, n) => acc + n.calificacion, 0);
        return suma / est.notas.length;
    }
    
    getEstadisticas() {
        const stats = {
            total: this.#estudiantes.size,
            promedios: [],
            aprobados: 0
        };
        
        this.#estudiantes.forEach((est, id) => {
            const prom = this.calcularPromedio(id);
            stats.promedios.push({id, nombre: est.nombre, promedio: prom.toFixed(2)});
            if (prom >= 60) stats.aprobados++;
        });
        
        return stats;
    }
    
    // Exportar para serializaci�n (no expone el Map directamente)
    toJSON() {
        return Array.from(this.#estudiantes.values());
    }
}

// Uso
const sistema = new RegistroAcademico();
sistema.agregarEstudiante('E001', 'Ana Garc�a', 'Sistemas');
sistema.agregarEstudiante('E002', 'Luis Torres', 'Industrial');

sistema.agregarNota('E001', 'Programaci�n', 85);
sistema.agregarNota('E001', 'Matem�ticas', 92);
sistema.agregarNota('E002', 'Programaci�n', 55);

const stats = sistema.getEstadisticas();

({
    "application/json": {
        sistema: "Registro Acad�mico v1.0",
        estadisticas: stats,
        data_completa: sistema.toJSON(),
        principio: "SRP: Separaci�n de datos, l�gica y presentaci�n"
    }
});

In [ ]:
// O - Open/Closed: Extender comportamiento sin modificar c�digo existente
// Strategy Pattern para c�lculo de becas

class CalculadoraBeca {
    #estrategia;
    
    constructor(estrategiaCalculo) {
        this.#estrategia = estrategiaCalculo;
    }
    
    calcular(montoBase, promedio) {
        return this.#estrategia.calcular(montoBase, promedio);
    }
}

// Estrategias intercambiables
const EstrategiaMerito = {
    calcular: (base, prom) => prom >= 90 ? base * 0.8 : base * 0.2
};

const EstrategiaDeportista = {
    calcular: (base, prom) => prom >= 70 ? base * 0.5 : 0
};

const EstrategiaNecesidad = {
    calcular: (base, prom) => base * 0.9 // 90% beca sin importar promedio
};

// Uso
const becaMerito = new CalculadoraBeca(EstrategiaMerito);
const becaDeporte = new CalculadoraBeca(EstrategiaDeportista);

({
    "text/html": `
    <h4>Strategy Pattern: C�lculo de Becas</h4>
    <ul>
        <li>Estudiante A (Prom 95, Estrategia M�rito): ${becaMerito.calcular(10000, 95)}</li>
        <li>Estudiante B (Prom 95, Estrategia Deportista): ${becaDeporte.calcular(10000, 95)}</li>
        <li>Estudiante C (Prom 80, Estrategia M�rito): ${becaMerito.calcular(10000, 80)}</li>
    </ul>
    <p><em>OCP: Nuevas estrategias sin modificar CalculadoraBeca</em></p>
    `
});